In [10]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [2]:
train = pd.read_csv('../data/train_final.csv')
test = pd.read_csv('../data/test_final.csv')

In [3]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'FamilySize',
       'FamilyName', 'Title', 'FirstName', 'FamilyNamePclass',
       'NDiedPerFamilyNamePclass', 'NSurvivedPerFamilyNamePclass',
       'DiffSurvivedDiedPerFamilyNamePclass', 'AgeImputedByPclassAndSex',
       'EmbarkedImputedByPclassAndSex', 'AgeClassPerCut4Bins',
       'AgeClassPerQCut4Quartiles', 'AgeClassPerFaixaEtaria',
       'AgeClassPerChuto'],
      dtype='object')

In [9]:
features = ['Pclass', 'AgeImputedByPclassAndSex', 'Fare', 'Sex', 'FamilySize']

train_X = pd.get_dummies(train[features])
train_y = train["Survived"]

test_X = pd.get_dummies(test[features])
# TODO: mover para cleansing
test_X.Fare.fillna(train_X.Fare.mean(), inplace=True)

age_scaller = MinMaxScaler()
age_scaller.fit(train_X.AgeImputedByPclassAndSex)

train_X['AgeImputedByPclassAndSex'] = age_scaller.transform(train_X.AgeImputedByPclassAndSex)
test_X['AgeImputdByPclassAndSex'] = age_scaller.transform(test_X.AgeImputedByPclassAndSex)

fare_scaller = MinMaxScaler()
fare_scaller.fit(train_X.Fare)

train_X['Fare'] = fare_scaller.transform(train_X.Fare)
test_X['Fare'] = fare_scaller.transform(test_X.Fare)

C:\Users\abevieira\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\abevieira\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\abevieira\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has 

In [11]:
rf = RandomForestClassifier(random_state = 1, n_jobs=-1)

param_grid_rf = {'max_features': ['auto', .2, .9],\
              'min_samples_leaf': [1, 2, 3],\
              'n_estimators': [100, 500, 1000],\
              'criterion': ['gini', 'entropy']}

model_rf = GridSearchCV(rf, param_grid=param_grid_rf, cv=5)

In [12]:
model_rf.fit(train_X, train_y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': ['auto', 0.2, 0.9], 'min_samples_leaf': [1, 2, 3], 'n_estimators': [100, 500, 1000], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [13]:
model_rf.best_score_

0.84287317620650959

In [14]:
prediction = model_rf.predict(test_X)

ValueError: Number of features of the model must  match the input. Model n_features is 6 and  input n_features is 7 

In [ ]:
submission = pd.DataFrame({"Survived": prediction, "PassengerId": test["PassengerId"]})
submission.to_csv('../submissions/Only Numericals v6.csv', index=False)